In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

%matplotlib inline

In [2]:
!tree

.
├── benchmark_arima.csv
├── benchmark.csv
├── benchmarks-master
│   ├── bees-benchmark.ipynb
│   ├── dengue-benchmark-statsmodels.ipynb
│   ├── fogwater-benchmark-model.ipynb
│   ├── LICENSE
│   └── README.md
├── benchmarks-master.zip
├── Dengue-benchmark-ARIMA.ipynb
├── dengue_features_test.csv
├── dengue_features_train.csv
├── dengue_labels_train.csv
├── dengue-notebook.ipynb
├── dengue-notebook-Regressions.ipynb
├── Iquitos-plaza.jpg
├── random_forest_regression.csv
├── README.md
├── results.csv
├── results_lasso.csv
├── results_RandomForestRegression.csv
├── results_Ridge.csv
├── San_Juan-Puerto_Rico.JPG
└── submission_format.csv

1 directory, 23 files


# Data import

## Preprocessing of the data

In [3]:
# load the provided data
train_features = pd.read_csv('dengue_features_train.csv')
train_labels = pd.read_csv('dengue_labels_train.csv')

In [4]:

train_features.columns

Index(['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm'],
      dtype='object')

In [5]:
def add_lag(dataframe, columns,number_of_lag):
    
    for column in columns: # for each feature (column)
        for lag in range(number_of_lag): # for each time step (lag)
            dataframe[column + "_time - "+ str(lag)] = dataframe[column].shift(lag) # copy the previous value
            
    #remove the first number_of_lag rows
    dataframe.fillna(method='backfill', inplace=True)
    
    return dataframe

def preprocess_data(data_path, labels_path=None):
    lag_step = 20
    
    # load data and set index to city, year, weekofyear
    df = pd.read_csv(data_path, index_col=[0, 1, 2])
    
    # select features we want
    features =['ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 
       'reanalysis_avg_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k', 'station_precip_mm']
    
    df = df[features]
    
    # fill missing values
    df.fillna(method='ffill', inplace=True)

    # add labels to dataframe
    if labels_path:
        labels = pd.read_csv(labels_path, index_col=[0, 1, 2])
        df = df.join(labels)
    
    # separate san juan and iquitos
    sj = df.loc['sj']
    iq = df.loc['iq']
    
    #add lag
    sj = add_lag(sj, features, lag_step)
    iq = add_lag(iq, features, lag_step)
    
    # fill navalues
    sj.fillna(method='backfill', inplace=True)
    iq.fillna(method='backfill', inplace=True)
    
    return sj, iq

In [6]:
sj_train, iq_train = preprocess_data('dengue_features_train.csv',
                                    labels_path="dengue_labels_train.csv")



/home/louis/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/louis/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2852: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


## Training 

In [7]:
from sklearn.ensemble import RandomForestRegressor
from math import floor

In [8]:
#from sklearn.model_selection import TimeSeriesSplit
#>>> X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
#>>> y = np.array([1, 2, 3, 4])
#>>> tscv = TimeSeriesSplit(n_splits=3)
#>>> print(tscv)  
#TimeSeriesSplit(max_train_size=None, n_splits=3)
#>>> for train_index, test_index in tscv.split(X):
#...    print("TRAIN:", train_index, "TEST:", test_index)
#...    X_train, X_test = X[train_index], X[test_index]
#...    y_train, y_test = y[train_index], y[test_index]
#TRAIN: [0] TEST: [1]
#TRAIN: [0 1] TEST: [2]
#TRAIN: [0 1 2] TEST: [3]

In [9]:
sj_features  = sj_train.drop('total_cases', axis =1)
sj_targets = sj_train['total_cases']

iq_features  = iq_train.drop('total_cases', axis =1)
iq_targets = iq_train['total_cases']

def split_time_series(X, y, test_set_ratio):
    

    number_of_examples = X.shape[0]
    last_index_in_train_set = floor( (1-test_set_ratio)*number_of_examples )
    
    X_train = X[:last_index_in_train_set]
    X_test = X[last_index_in_train_set:]
    
    y_train = y[:last_index_in_train_set]
    y_test = y[last_index_in_train_set:]
    
    return X_train, X_test, y_train, y_test


test_set_ratio = 0.3 

X_sj_train, X_sj_test, y_sj_train, y_sj_test = split_time_series(sj_features, sj_targets, test_set_ratio)
X_iq_train, X_iq_test, y_iq_train, y_iq_test = split_time_series(iq_features, iq_targets, test_set_ratio)


In [ ]:
from sklearn.metrics import mean_absolute_error

# for San Juan
def find_best_params(X_train,y_train,X_val,y_val,n_estimators,max_depths):
    
    print("mean_absolute_error [n_estomators,max_depth]: mae")
    scores_mean_absolute_error = np.zeros((len(n_estimators), len(max_depths) ))
    best_score_mean_absolute_error = float("inf")
    best_parameters = [0,0]
    
    for i  in range(len(n_estimators)):
        for j in range(len(max_depths)):

            n_estimator = n_estimators[i]
            max_depth = max_depths[j]

            regressor_RF = RandomForestRegressor(n_estimators=n_estimator,max_depth = max_depth, random_state=0, n_jobs=8)
            regressor_RF.fit(X_train,y_train)
            predictions = regressor_RF.predict(X_val)

            mean_absolute_error_ = mean_absolute_error(y_val, predictions)
            print("mean_absolute_error [{0},{1}]: {2}".format(n_estimator,max_depth,mean_absolute_error_))
            scores_mean_absolute_error[i,j] = mean_absolute_error_

            if mean_absolute_error_ < best_score_mean_absolute_error:
               best_score_mean_absolute_error = mean_absolute_error_
               best_parameters = [n_estimator, max_depth] 
    
        

    print("\n-------------------------")
    print("best_mean_absolute_error: {0}".format(best_score_mean_absolute_error))         
    print("n_estimators: {0}".format(best_parameters[0]))         
    print("max_depth: {0}".format(best_parameters[1])) 
    print("-------------------------\n")
    
    best_n_estimators = best_parameters[0]
    best_max_depth = best_parameters[1]
    
    return best_n_estimators, best_max_depth,scores_mean_absolute_error

In [ ]:
n_estimators = [20,100,400,800,1200]
max_depths = [15,20, 40]

n_estimators_sj, max_depth_sj, scores__mean_absulute_error_sj = find_best_params(X_sj_train,
                                                                                 y_sj_train,
                                                                                 X_sj_test,
                                                                                 y_sj_test,
                                                                                 n_estimators,
                                                                                 max_depths)

mean_absolute_error [n_estomators,max_depth]: mae
mean_absolute_error [20,15]: 29.845397749158924
mean_absolute_error [20,20]: 30.089825434478982
mean_absolute_error [20,40]: 30.319572953736653
mean_absolute_error [100,15]: 31.274181070022884
mean_absolute_error [100,20]: 31.11010967837402
mean_absolute_error [100,40]: 31.12288256227758
mean_absolute_error [400,15]: 30.972119275235553


In [ ]:
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores__mean_absulute_error_sj, interpolation='nearest', cmap=plt.cm.hot_r)
plt.ylabel('n_estimators')
plt.xlabel('max_depth')
plt.colorbar()
plt.yticks(np.arange(len(n_estimators)), n_estimators, rotation=45)
plt.xticks(np.arange(len(max_depths)), max_depths)
plt.title('Validation accuracy for San Juan')
plt.show()

In [ ]:
n_estimators_iq, max_depth_iq, scores__mean_absulute_error_iq = find_best_params(X_iq_train,
                                                                                 y_iq_train,
                                                                                 X_iq_test,
                                                                                 y_iq_test,
                                                                                 n_estimators,
                                                                                 max_depths)

In [ ]:
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores__mean_absulute_error_iq, interpolation='nearest', cmap=plt.cm.hot_r)
plt.ylabel('n_estimators')
plt.xlabel('max_depth')
plt.colorbar()
plt.yticks(np.arange(len(n_estimators)), n_estimators, rotation=45)
plt.xticks(np.arange(len(max_depths)), max_depths)
plt.title('Validation accuracy for Iquitos')
plt.show()

In [ ]:
regressor_RF_sj = RandomForestRegressor(n_estimators = n_estimators_sj ,
                                        max_depth=max_depth_sj,
                                        random_state=101,
                                        n_jobs=8)

regressor_RF_iq = RandomForestRegressor(n_estimators=n_estimators_iq ,
                                        max_depth=max_depth_iq,
                                        random_state=101,
                                        n_jobs=8)

In [ ]:
regressor_RF_sj.fit(X_sj_train,y_sj_train)
regressor_RF_iq.fit(X_iq_train,y_iq_train)

In [ ]:
predictions_sj = regressor_RF_sj.predict(X_sj_test)
predictions_iq = regressor_RF_iq.predict(X_iq_test)

In [ ]:
from sklearn.metrics import mean_squared_error

print("For San Juan\n ")
print("mean_squared_error: {0}".format(mean_squared_error(y_sj_test, predictions_sj)))
print("mean_absolute_error: {0}".format(mean_absolute_error(y_sj_test, predictions_sj)))
print("\nFor Iquitos\n ")
print("mean_squared_error: {0}".format(mean_squared_error(y_iq_test, predictions_iq)))
print("mean_absolute_error: {0}".format(mean_absolute_error(y_iq_test, predictions_iq)))

df1 = pd.DataFrame({"predicted":predictions_sj,"actual":y_sj_test})

df1.plot()
plt.title('San Juan: predicted vs actual')
plt.xlabel('Time')

df2 = pd.DataFrame({"predicted":predictions_iq,"actual":y_iq_test})

df2.plot()
plt.title('Iquitos: predicted vs actual')
plt.xlabel('Time')

plt.legend()

In [ ]:
regressor_RF_sj_final = RandomForestRegressor(n_estimators = n_estimators_sj ,
                                        max_depth=max_depth_sj,
                                        random_state=101,
                                        n_jobs=8)

regressor_RF_iq_final = RandomForestRegressor(n_estimators=n_estimators_iq ,
                                        max_depth=max_depth_iq,
                                        random_state=101,
                                        n_jobs=8)

regressor_RF_sj.fit(sj_features,sj_targets)
regressor_RF_iq.fit(iq_features,iq_targets)

In [ ]:
sj_test, iq_test = preprocess_data('dengue_features_test.csv')



sj_predictions = regressor_RF_sj.predict(sj_test).astype(int)
iq_predictions = regressor_RF_iq.predict(iq_test).astype(int)

submission = pd.read_csv("submission_format.csv",
                         index_col=[0, 1, 2])

submission.total_cases = np.concatenate((sj_predictions, iq_predictions),axis=0)
submission.to_csv("random_forest_regression.csv")